In [1]:
import pandas as pd
import boto3
import io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Set the maximum number of rows and Columns to 200
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)

In [2]:
s3R = boto3.resource("s3")
s3 = boto3.client("s3")
bucket_name = "capstonehaystacks"
# Listing the contents of the S3 bucket
for obj in s3R.Bucket(bucket_name).objects.all():
    print(obj.key)

ACSDP5Y2021.DP04-Data.csv
ACSST5Y2021.S0101-Data.csv
ACSST5Y2021.S0801-Data.csv
ACSST5Y2021.S1901-Data.csv
GA_LISTINGS_SALES.csv
GA_LISTINGS_SALES_V2.csv
GA_listing.csv
all_zips_grocery_store.json
all_zips_restaurant.json
atlanta-geo.csv
atlanta_cbsa_zip.csv
atlanta_hdma_2021.csv
atlanta_hdma_census_2021.csv
atlanta_hdma_tract_2021.csv
atlanta_listings.csv
census_all.csv
census_all_perCapita.csv
census_data
core_geo_dataset.csv
crime.csv
crime_rating_zipcode.csv
elementary_schools.csv
ga_georgia_zip_codes_geo.min.json
high_schools.csv
hmda_2017_ga_all-records_labels.csv
hmda_2021_ga_all-records_labels.csv
jason_listing.csv
listing-with-poi-distances.csv
listings_with_tract.csv
middle_schools.csv
poi-google-exploded.csv
poi_combined_haystack.csv
poi_combined_haystack_ALL.csv
poi_combined_haystack_ALL_CLEANED.csv
poi_expanded.csv
points-of-interest-google.csv
points-of-interest-google2.csv
points-of-interest-google3.csv
points-of-interest-haystacks.csv
schools.csv
zip_summary.csv
zipcode

In [3]:
file_name = "census_all.csv"

# Downloading the file from S3 and parsing it into a dataframe
s3.download_file(bucket_name, file_name, file_name)

# Low memory is set to False because columns have mixed data types
census = pd.read_csv(file_name, index_col=False, low_memory=False)

In [4]:
file_name = "census_all_perCapita.csv"

# Downloading the file from S3 and parsing it into a dataframe
s3.download_file(bucket_name, file_name, file_name)

# Low memory is set to False because columns have mixed data types
census_capita = pd.read_csv(file_name, index_col=False, low_memory=False)

In [5]:
file_name = "atlanta_hdma_2021.csv"

# Downloading the file from S3 and parsing it into a dataframe
s3.download_file(bucket_name, file_name, file_name)

# Low memory is set to False because columns have mixed data types
hdma = pd.read_csv(file_name, index_col=False, low_memory=False)

In [6]:
census_capita.shape

(207, 800)

In [7]:
# filling missing values with 0 which are the restaurants above 4.7 rating and below 4.7 rating
census_capita = census_capita.fillna(0)

In [8]:
# renaming first column
hdma = hdma.rename(columns={hdma.columns[0]: 'zipcode'})

In [10]:
# merging hdma data to census capita
merged_df = pd.merge(census_capita, hdma[['zipcode', 'approval_percentage']], on='zipcode', how='left', suffixes=('', '_hdma'))



In [12]:
# moving columns positions

# list of columns to move them
cols = list(merged_df.columns)

# moving column at index 801 to index 1
cols.insert(1, cols.pop(800))

# reindexing
merged_df = merged_df[cols]

In [13]:
merged_df

,zipcode,approval_percentage,restaurant_above_4.7_percentage,total_workers,car_commute,travel_less_10,travel_10_14,travel_15_19,travel_20_24,travel_25_29,travel_30_34,travel_35_44,travel_45_59,travel_more_60,travel_mean,population,percent_male,percent_under_15,percent_teen_15_19,percent_college_20_24,percent_25_39,percent_40-59,percent_over_60,total_units,rental_vacancy_rate,median_homeowner_value,median_rental_value,percent_owner_occupied,percent_after_2019,rent_less_15_percent_income,rent_over_30_percent,rent_15_30_percent,rent_less_999,rent_1000_2500,rent_over_2500,gross_rental_yield,total_households,percent_less_10k,percent_10k_15k,percent_15k_25k,percent_25k_35k,percent_35k_50k,percent_50k_75k,percent_75k_100k,percent_100k_150k,percent_150k_200k,percent_more_200k,household_median_income,household_mean_income,accountant_per_Capita,addiction_treatment_center_per_Capita,advertising_agency_per_Capita,after_school_program_per_Capita,air_conditioning_contractor_per_Capita,air_conditioning_repair_service_per_Capita,air_conditioning_system_supplier_per_Capita,air_duct_cleaning_service_per_Capita,allergist_per_Capita,alternative_fuel_station_per_Capita,alternative_medicine_practitioner_per_Capita,american_restaurant_per_Capita,amusement_center_per_Capita,animal_control_service_per_Capita,animal_hospital_per_Capita,antique_furniture_store_per_Capita,antique_store_per_Capita,apartment_building_per_Capita,apartment_complex_per_Capita,apartment_rental_agency_per_Capita,appliance_repair_service_per_Capita,appliance_store_per_Capita,art_center_per_Capita,art_gallery_per_Capita,asian_fusion_restaurant_per_Capita,asian_restaurant_per_Capita,assisted_living_facility_per_Capita,association_or_organization_per_Capita,atm_per_Capita,attorney_per_Capita,audiologist_per_Capita,auto_air_conditioning_service_per_Capita,auto_body_parts_supplier_per_Capita,auto_body_shop_per_Capita,auto_broker_per_Capita,auto_dent_removal_service_per_Capita,auto_electrical_service_per_Capita,auto_glass_shop_per_Capita,auto_insurance_agency_per_Capita,auto_parts_store_per_Capita,auto_radiator_repair_service_per_Capita,auto_repair_shop_per_Capita,auto_restoration_service_per_Capita,auto_tune_up_service_per_Capita,auto_wrecker_per_Capita,automobile_storage_facility_per_Capita,baby_clothing_store_per_Capita,baby_store_per_Capita,bagel_shop_per_Capita,bail_bonds_service_per_Capita,bakery_per_Capita,...,taco_restaurant_per_Capita,tailor_per_Capita,takeout_restaurant_per_Capita,tanning_salon_per_Capita,tattoo_shop_per_Capita,tax_consultant_per_Capita,tax_preparation_per_Capita,tax_preparation_service_per_Capita,tea_house_per_Capita,teeth_whitening_service_per_Capita,telecommunications_service_provider_per_Capita,telephone_company_per_Capita,temp_agency_per_Capita,tennis_court_per_Capita,tex-mex_restaurant_per_Capita,thai_restaurant_per_Capita,thrift_store_per_Capita,tile_store_per_Capita,tire_shop_per_Capita,tobacco_shop_per_Capita,tool_rental_service_per_Capita,tool_store_per_Capita,topsoil_supplier_per_Capita,tourist_attraction_per_Capita,towing_service_per_Capita,townhouse_complex_per_Capita,toy_store_per_Capita,traditional_american_restaurant_per_Capita,trailer_dealer_per_Capita,trailer_rental_service_per_Capita,trailer_repair_shop_per_Capita,trailer_supply_store_per_Capita,transmission_shop_per_Capita,transportation_service_per_Capita,travel_agency_per_Capita,tree_service_per_Capita,trial_attorney_per_Capita,truck_accessories_store_per_Capita,truck_dealer_per_Capita,truck_parts_supplier_per_Capita,truck_rental_agency_per_Capita,truck_repair_shop_per_Capita,truck_stop_per_Capita,trucking_company_per_Capita,tuxedo_shop_per_Capita,uniform_store_per_Capita,united_methodist_church_per_Capita,university_per_Capita,upholstery_cleaning_service_per_Capita,upholstery_shop_per_Capita,urgent_care_center_per_Capita,urologist_per_Capita,used_auto_parts_store_per_Capita,used_car_dealer_per_Capita,used_tire_shop_per_Capita,used_truck_dealer_per_Capita,van_rental_agency_per_Capita,va

In [ ]:
# looking at column names and corrosponding numbers

for idx, column_name in enumerate(merged_df.columns):
    print(f"Column {idx}: {column_name}")

In [ ]:
# dropping a few columns that are not represented in percentages such as population and income
merged_df = merged_df.drop(merged_df.columns[[2, 3, 15, 23, 25, 26, 36, 47, 48, ]], axis=1)

In [ ]:
# making the zip code column the index
merged_df.set_index(merged_df.columns[0], inplace=True)


In [ ]:
merged_df.describe()


In [ ]:
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_rows", 200)

In [ ]:
correlation = merged_df.corr()['gross_rental_yield']
correlation.sort_values(ascending=False)

In [ ]:
correlation = merged_df.corr()['approval_percentage']
correlation.sort_values(ascending=False)


## Transforming data

In [ ]:
# converting the first 29 columns into percentages by dividing by 100
columns_to_divide = list(range(0, 39))
merged_df.iloc[:, columns_to_divide] = merged_df.iloc[:, columns_to_divide] / 100


In [ ]:
# adding a new column with total poi's per 1000
merged_df['total_poi_per_1000'] = merged_df.iloc[:, 39:791].sum(axis=1)


In [ ]:
# converting POI columns to percent of total. 

columns_to_convert = list(range(39, 791))
total_sum = merged_df.iloc[:, columns_to_convert].sum(axis=1)
merged_df.iloc[:, columns_to_convert] = merged_df.iloc[:, columns_to_convert].div(total_sum, axis=0)


In [ ]:
# dropping last column
merged_df = merged_df.drop(columns=merged_df.columns[-1])


In [ ]:
pd.set_option("display.max_rows", 200)
merged_df.dtypes

In [ ]:
# looking into POI's relationship to zip code features

features = merged_df.iloc[:, 0:40]  # Select columns 0-39
target = merged_df['approval_percentage']



# training the linear regression model and evaluate for each column
for col in features.columns:
    X_train, X_test, y_train, y_test = train_test_split(features[col].values.reshape(-1, 1), target, test_size=0.2, random_state=42)

    regression_model = LinearRegression()
    regression_model.fit(X_train, y_train)

    score = regression_model.score(X_test, y_test)
    print(f"Column {col} - R-squared: {score}")


In [ ]:
# seperating POI information from census info

target_list = [
    "approval_percentage",
    "car_commute",
    "travel_less_10",
    "travel_10_14",
    "travel_15_19",
    "travel_20_24",
    "travel_25_29",
    "travel_30_34",
    "travel_35_44",
    "travel_45_59",
    "travel_more_60",
    "travel_mean",
    "percent_male",
    "percent_under_15",
    "percent_teen_15_19",
    "percent_college_20_24",
    "percent_25_39",
    "percent_40-59",
    "percent_over_60",
    "rental_vacancy_rate",
    "percent_owner_occupied",
    "percent_after_2019",
    "rent_less_15_percent_income",
    "rent_over_30_percent",
    "rent_15_30_percent",
    "rent_less_999",
    "rent_1000_2500",
    "rent_over_2500",
    "gross_rental_yield",
    "percent_less_10k",
    "percent_10k_15k",
    "percent_15k_25k",
    "percent_25k_35k",
    "percent_35k_50k",
    "percent_50k_75k",
    "percent_75k_100k",
    "percent_100k_150k",
    "percent_150k_200k",
    "percent_more_200k"
]


## looking into correlations of POI data with target_list

In [ ]:

# list of columns not in target list
other_columns = [column for column in merged_df.columns if column not in target_list]

# correlation between target_list and other_columns
correlation_results = {}

for target_column in target_list:
    correlation_results[target_column] = {}
    for other_column in other_columns:
        correlation = merged_df[target_column].corr(merged_df[other_column])
        correlation_results[target_column][other_column] = correlation

# column with the highest correlation for each column in target_list
highest_correlation = []
for target_column in target_list:
    max_corr_column = max(correlation_results[target_column], key=correlation_results[target_column].get)
    max_corr_value = correlation_results[target_column][max_corr_column]
    highest_correlation.append((target_column, max_corr_column, max_corr_value))

# column with the lowest correlation for each column in target_list
lowest_correlation = []
for target_column in target_list:
    min_corr_column = min(correlation_results[target_column], key=correlation_results[target_column].get)
    min_corr_value = correlation_results[target_column][min_corr_column]
    lowest_correlation.append((target_column, min_corr_column, min_corr_value))

# sorting
highest_correlation.sort(key=lambda x: x[2], reverse=True)
lowest_correlation.sort(key=lambda x: x[2], reverse=True)

# printing results
print("Columns in target_list with highest correlation:")
for pair in highest_correlation:
    print(pair[0], "to", pair[1], "- Correlation:", pair[2])

print("\nColumns in target_list with lowest correlation:")
for pair in lowest_correlation:
    print(pair[0], "to", pair[1], "- Correlation:", pair[2])


In [ ]:


# list of columns not in target lists
other_columns = [column for column in merged_df.columns if column not in target_list]

# correlation between columns in target_list and other_columns
correlation_results = {}

for target_column in target_list:
    correlation_results[target_column] = {}
    for other_column in other_columns:
        correlation = merged_df[target_column].corr(merged_df[other_column])
        if abs(correlation) > 0.5:
            correlation_results[target_column][other_column] = correlation

# printing the columns with correlations above 0.6 or below -0.6
for target_column in correlation_results:
    print("Correlations for", target_column)
    for other_column, correlation in correlation_results[target_column].items():
        print(target_column, "to", other_column, "- Correlation:", correlation)


## Seperating census data from POI data

In [ ]:
# dataframe with columns in target_list
target_df = merged_df[target_list]

# dataframe with columns not in target_list
features_df = merged_df.drop(target_list, axis=1)

# KNN

In [ ]:
# Creating categories from continous data

# histogram of approval percentage
plt.hist(target_df['approval_percentage'], bins=10, edgecolor='black')

# labels and title
plt.xlabel('Approval Percentage')
plt.ylabel('Frequency')
plt.title('Distribution of Approval Percentage')


plt.show()


In [ ]:

# defining the category ranges and names
category_ranges = [0, 0.8, 0.85, 0.9, 1]
category_names = ['approv_below_80', 'approv_80_85', 'approv_85_90', 'approve_above_90']

# creating copy of the dataframe and assign category labels based on ranges
target_df_copy = target_df.copy()
target_df_copy['approval_category'] = pd.cut(target_df_copy['approval_percentage'], bins=category_ranges, labels=category_names)


In [ ]:
# KNN

X = features_df  # POI columns using to predict target
y = target_df_copy['approval_category']  # new categorical target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3)  # creating an instance of KNeighborsClassifier
knn.fit(X_train_scaled, y_train)  # fitting the model to the training data

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1) # zero_division equal to one to prevent errors
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)


In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
# Creating categories from continous data

# histogram of approval percentage
plt.hist(target_df['gross_rental_yield'], bins=10, edgecolor='black')

# labels and title
plt.xlabel('Gross Rental Yield')
plt.ylabel('Frequency')
plt.title('Distribution of Gross Rental Yield')


plt.show()



In [ ]:
# creating categories from continous data

# defining the category ranges and names
category_ranges = [0, 0.04, 0.06, 0.08, float('inf')]
category_names = ['yield_below_0.04', 'yield_0.04_0.06', 'yield_0.06_0.08', 'yield_above_0.08']

# creating a new categorical column based on the 'gross_rental_yield' column
target_df_copy = target_df.copy()
target_df_copy['yield_category'] = pd.cut(target_df_copy['gross_rental_yield'], bins=category_ranges, labels=category_names)


In [ ]:
X = features_df  
y = target_df_copy['yield_category']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3)  
knn.fit(X_train_scaled, y_train)  

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

report = classification_report(y_test, y_pred, zero_division=1)
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)


In [ ]:
car_commute

In [ ]:
import matplotlib.pyplot as plt

# Plotting the histogram
plt.hist(target_df['car_commute'], bins=10, edgecolor='black')

# Adding labels and title
plt.xlabel('Car Commute')
plt.ylabel('Frequency')
plt.title('Distribution of Percentage Car Commuting')

# Display the histogram
plt.show()


In [ ]:

# creating equal bins for categorical commuting data
num_bins = 4

# making equal size bin boundaries
bin_boundaries = np.linspace(target_df_copy['car_commute'].min(), target_df_copy['car_commute'].max(), num_bins + 1)

# defining the category names based on range
category_names = [f'commute_{bin_boundaries[i]:.2f}_{bin_boundaries[i+1]:.2f}' for i in range(num_bins)]

# creating a copy of the dataframe and assign category labels based on equal size bins
target_df_copy['car_commute_category'] = pd.cut(target_df_copy['car_commute'], bins=bin_boundaries, labels=category_names)


In [ ]:
# filling in a missing value
target_df_copy['car_commute_category'] = target_df_copy['car_commute_category'].fillna('commute_0.22_0.42')


# I think below is what we want

In [ ]:
# from sklearn.neighbors import NearestNeighbors
# from sklearn.preprocessing import StandardScaler

# X = features_df  # Your feature columns
# y = target_df_copy['car_commute_category']  # The new categorical target column

# # Assuming you have a single target row for which you want to find similar rows
# target_row = X.iloc[0].values  # Assuming the first row as the target and extracting the values

# # Scale the data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Calculate distances to find similar rows
# neigh = NearestNeighbors(n_neighbors=5, metric='euclidean')  # Choose the value of K and distance metric
# neigh.fit(X_scaled)
# distances, indices = neigh.kneighbors(scaler.transform([target_row]))  # Pass target_row as a list

# similar_rows = X.iloc[indices[0]]

# print("Target Row:")
# print(target_row)
# print("\nSimilar Rows:")
# print(similar_rows)



In [ ]:
target_row

In [ ]:
X = features_df  # Your feature columns
y = target_df_copy['car_commute_category']  # The new categorical target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3)  # Create an instance of KNeighborsClassifier
knn.fit(X_train_scaled, y_train)  # Fit the model to the training data

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

In [ ]:
# creating histogram
plt.hist(target_df['percent_more_200k'], bins=10, edgecolor='black')

# labels and title
plt.xlabel('percent_more_200k')
plt.ylabel('Frequency')
plt.title('Distribution of percent_more_200k')

plt.show()

In [ ]:


# category ranges and names
category_ranges = [0.0, 0.0911, 0.211, 1]
category_names = ['below_.09', 'between.09_.2',  'above_0.2']

# creating a new categorical column based on the 'gross_rental_yield' column
target_df_copy = target_df.copy()
target_df_copy['percent_more_200k_category'] = pd.cut(target_df_copy['percent_more_200k'], bins=category_ranges, labels=category_names)


In [ ]:
# fillling in 3 missing values
target_df_copy['percent_more_200k_category'] = target_df_copy['percent_more_200k_category'].fillna('below_.09')


In [ ]:
X = features_df  
y = target_df_copy['percent_more_200k_category']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(X_train_scaled, y_train)  

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

In [ ]:
target_df_copy.isna().sum()